<a href="https://colab.research.google.com/github/Siva-Sai-B/hello-world/blob/master/Network_Slicing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install swig -y
!pip install Cython numpy

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  swig3.0
Suggested packages:
  swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  swig swig3.0
0 upgraded, 2 newly installed, 0 to remove and 20 not upgraded.
Need to get 1,100 kB of archives.
After this operation, 5,822 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig3.0 amd64 3.0.12-1 [1,094 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig amd64 3.0.12-1 [6,460 B]
Fetched 1,100 kB in 1s (1,355 kB/s)
Selecting previously unselected package swig3.0.
(Reading database ... 155569 files and directories currently installed.)
Preparing to unpack .../swig3.0_3.0.12-1_amd64.deb ...
Unpa

In [ ]:
!pip install auto-sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.4 MB 31.5 MB/s 
     |████████████████████████████████| 22.3 MB 1.8 MB/s 
     |████████████████████████████████| 4.3 MB 60.6 MB/s 
     |████████████████████████████████| 4.4 MB 69.3 MB/s 
     |████████████████████████████████| 260 kB 75.4 MB/s 
     |████████████████████████████████| 46 kB 2.4 MB/s 
  Created wheel for auto-sklearn: filename=auto_sklearn-0.14.7-py3-none-any.whl size=6602873 sha256=53c466655e696b7e5b5f2ee02a2902ca46fc36f879a203ce3b7da327a96c7bf6
  Stored in directory: /root/.cache/pip/wheels/ba/43/5c/2fbe6fd19e3af314cbc4aa808378068d8ddd6792064f4a2448
  Created wheel for pynisher: filename=pynisher-0.6.4-py3-none-any.whl size=7043 sha256=f644e573b57f531565d4fb8d1c4d121aa220c2844e620fd162c768c1b422e0af
  Stored in directory: /root/.cache/pip/wheels/42/71/95/7555ec3253e1ba8add72ae5febf1b015d297f3b73ba296d6f6
  Created wheel for sma

In [ ]:
import sklearn
import autosklearn.classification as classifier
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import pandas as pd
train=pd.read_csv('/content/drive/MyDrive/train_dataset.csv')

In [3]:
train.head()


,LTE/5g Category,Time,Packet Loss Rate,Packet delay,IoT,LTE/5G,GBR,Non-GBR,AR/VR/Gaming,Healthcare,Industry 4.0,IoT Devices,Public Safety,Smart City & Home,Smart Transportation,Smartphone,slice Type
0,14,0,0.000001,10,1,0,0,1,0,0,0,0,1,0,0,0,3
1,18,20,0.001000,100,0,1,1,0,1,0,0,0,0,0,0,0,1
2,17,14,0.000001,300,0,1,0,1,0,0,0,0,0,0,0,1,1
3,3,17,0.010000,100,0,1,0,1,0,0,0,0,0,0,0,1,1
4,9,4,0.010000,50,1,0,0,1,0,0,0,0,0,1,0,0,2


In [4]:
test=pd.read_csv('/content/drive/MyDrive/test_dataset.csv')
test.head()

,LTE/5g Category,Time,Packet Loss Rate,Packet delay,IoT,LTE/5G,GBR,Non-GBR,AR/VR/Gaming,Healthcare,Industry 4.0,IoT Devices,Public Safety,Smart City & Home,Smart Transportation,Smartphone
0,15,17,0.001000,100,0,1,1,0,1,0,0,0,0,0,0,0
1,14,18,0.000001,10,1,0,0,1,0,0,0,0,0,0,1,0
2,11,7,0.001000,50,1,0,1,0,0,0,1,0,0,0,0,0
3,20,14,0.001000,50,1,0,1,0,0,0,1,0,0,0,0,0
4,2,22,0.001000,50,0,1,0,1,1,0,0,0,0,0,0,0


In [5]:
def reduce_mem_usage(train):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = train.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in train.columns:
        col_type = train[col].dtype
        
        if col_type != object:
            c_min = train[col].min()
            c_max = train[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    train[col] = train[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    train[col] = train[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    train[col] = train[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    train[col] = train[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    train[col] = train[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    train[col] = train[col].astype(np.float32)
                else:
                    train[col] = train[col].astype(np.float64)
        else:
            train[col] = train[col].astype('object')

    end_mem = train.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return train

In [6]:
import numpy as np
train= reduce_mem_usage(train)
train.info()

Memory usage of dataframe is 4.10 MB
Memory usage after optimization is: 0.57 MB
Decreased by 86.0%
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31583 entries, 0 to 31582
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   LTE/5g Category       31583 non-null  int8   
 1   Time                  31583 non-null  int8   
 2   Packet Loss Rate      31583 non-null  float16
 3   Packet delay          31583 non-null  int16  
 4   IoT                   31583 non-null  int8   
 5   LTE/5G                31583 non-null  int8   
 6   GBR                   31583 non-null  int8   
 7   Non-GBR               31583 non-null  int8   
 8   AR/VR/Gaming          31583 non-null  int8   
 9   Healthcare            31583 non-null  int8   
 10  Industry 4.0          31583 non-null  int8   
 11  IoT Devices           31583 non-null  int8   
 12  Public Safety         31583 non-null  int8   
 13  Smart City & Home    

In [7]:
test= reduce_mem_usage(test)
test.info()

Memory usage of dataframe is 3.86 MB
Memory usage after optimization is: 0.54 MB
Decreased by 85.9%
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31584 entries, 0 to 31583
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   LTE/5g Category       31584 non-null  int8   
 1   Time                  31584 non-null  int8   
 2   Packet Loss Rate      31584 non-null  float16
 3   Packet delay          31584 non-null  int16  
 4   IoT                   31584 non-null  int8   
 5   LTE/5G                31584 non-null  int8   
 6   GBR                   31584 non-null  int8   
 7   Non-GBR               31584 non-null  int8   
 8   AR/VR/Gaming          31584 non-null  int8   
 9   Healthcare            31584 non-null  int8   
 10  Industry 4.0          31584 non-null  int8   
 11  IoT Devices           31584 non-null  int8   
 12  Public Safety         31584 non-null  int8   
 13  Smart City & Home    

In [8]:
y = train.pop('slice Type')
X = train

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42,shuffle=True, stratify=y)

In [ ]:
automlclassifier=classifier.AutoSklearnClassifier(time_left_for_this_task=180,per_run_time_limit=40)
automlclassifier.fit(X_train,y_train)

AutoSklearnClassifier(per_run_time_limit=40, time_left_for_this_task=180)

In [ ]:
automlclassifier.show_models()

{2: {'model_id': 2,
  'rank': 1,
  'cost': 0.0,
  'ensemble_weight': 0.08,
  'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice at 0x7f281aa2f190>,
  'balancing': Balancing(random_state=1),
  'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice at 0x7f281aa87f10>,
  'classifier': <autosklearn.pipeline.components.classification.ClassifierChoice at 0x7f281aa87bd0>,
  'sklearn_classifier': RandomForestClassifier(max_features=4, n_estimators=512, n_jobs=1,
                         random_state=1, warm_start=True)},
 4: {'model_id': 4,
  'rank': 2,
  'cost': 0.0,
  'ensemble_weight': 0.12,
  'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice at 0x7f281aa60490>,
  'balancing': Balancing(random_state=1, strategy='weighting'),
  'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice at 0x7f281aa98490>,
  

In [12]:
from sklearn.metrics import accuracy_score


In [ ]:
y_pred=automlclassifier.predict(X_test)

In [ ]:
print(accuracy_score(y_test,y_pred))

1.0


In [14]:
from sklearn.metrics import classification_report


In [ ]:
print(classification_report(y_train, automlclassifier.predict(X_train)))

In [ ]:
print(classification_report(y_test, automlclassifier.predict(X_test)))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00      3360
           2       1.00      1.00      1.00      1479
           3       1.00      1.00      1.00      1478

    accuracy                           1.00      6317
   macro avg       1.00      1.00      1.00      6317
weighted avg       1.00      1.00      1.00      6317



In [ ]:
y_pred = automlclassifier.predict(test)
y_pred[:20]


array([1, 3, 2, 2, 1, 1, 2, 2, 3, 1, 1, 1, 2, 1, 3, 3, 1, 1, 1, 2],
      dtype=int8)

In [ ]:
sol = pd.DataFrame(y_pred,columns=['slice Type'])
sol.head(20)

,slice Type
0,1
1,3
2,2
3,2
4,1
5,1
6,2
7,2
8,3
9,1


In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
RFC=RandomForestClassifier(max_features=4, n_estimators=512, n_jobs=1, random_state=1, warm_start=True)

In [ ]:
RFC.fit(X_train,y_train)

RandomForestClassifier(max_features=4, n_estimators=512, n_jobs=1,
                       random_state=1, warm_start=True)

In [ ]:
y_pred=RFC.predict(X_test)

In [ ]:
print(accuracy_score(y_test,y_pred))

1.0


In [ ]:
print(classification_report(y_train, RFC.predict(X_train)))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00     13439
           2       1.00      1.00      1.00      5913
           3       1.00      1.00      1.00      5914

    accuracy                           1.00     25266
   macro avg       1.00      1.00      1.00     25266
weighted avg       1.00      1.00      1.00     25266



In [ ]:
print(classification_report(y_test, RFC.predict(X_test)))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00      3360
           2       1.00      1.00      1.00      1479
           3       1.00      1.00      1.00      1478

    accuracy                           1.00      6317
   macro avg       1.00      1.00      1.00      6317
weighted avg       1.00      1.00      1.00      6317



In [ ]:
y_pred = RFC.predict(test)
y_pred[:20]


array([1, 3, 2, 2, 1, 1, 2, 2, 3, 1, 1, 1, 2, 1, 3, 3, 1, 1, 1, 2],
      dtype=int8)

In [ ]:
sol = pd.DataFrame(y_pred,columns=['slice Type'])
sol.head(20)

,slice Type
0,1
1,3
2,2
3,2
4,1
5,1
6,2
7,2
8,3
9,1


In [ ]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
HGBC= HistGradientBoostingClassifier(early_stopping=True,
                                 l2_regularization=1.3153561911035717e-10,
                                 learning_rate=0.025345400213312417, max_iter=512,
                                 max_leaf_nodes=16, min_samples_leaf=27,
                                 n_iter_no_change=5, random_state=1,
                                 validation_fraction=0.2361478213622299,
                                 warm_start=True)
HGBC.fit(X_train,y_train)
y_pred=HGBC.predict(X_test)
print(accuracy_score(y_test,y_pred))
print(classification_report(y_train, HGBC.predict(X_train)))
print(classification_report(y_test, HGBC.predict(X_test)))
y_pred = HGBC.predict(test)
print(y_pred[:20])
sol = pd.DataFrame(y_pred,columns=['slice Type'])
print(sol.head(20))

1.0
              precision    recall  f1-score   support

           1       1.00      1.00      1.00     13439
           2       1.00      1.00      1.00      5913
           3       1.00      1.00      1.00      5914

    accuracy                           1.00     25266
   macro avg       1.00      1.00      1.00     25266
weighted avg       1.00      1.00      1.00     25266

              precision    recall  f1-score   support

           1       1.00      1.00      1.00      3360
           2       1.00      1.00      1.00      1479
           3       1.00      1.00      1.00      1478

    accuracy                           1.00      6317
   macro avg       1.00      1.00      1.00      6317
weighted avg       1.00      1.00      1.00      6317

[1 3 2 2 1 1 2 2 3 1 1 1 2 1 3 3 1 1 1 2]
    slice Type
0            1
1            3
2            2
3            2
4            1
5            1
6            2
7            2
8            3
9            1
10           1
11          

In [ ]:
from sklearn.linear_model import SGDClassifier

SGDC= SGDClassifier(alpha=1.6992296128865824e-07, average=True, eta0=0.01, loss='log',
                max_iter=16, penalty='l1', random_state=1,
                tol=1.535384699341134e-05, warm_start=True)
SGDC.fit(X_train,y_train)
y_pred=SGDC.predict(X_test)
print(accuracy_score(y_test,y_pred))
print(classification_report(y_train, SGDC.predict(X_train)))
print(classification_report(y_test, SGDC.predict(X_test)))
y_pred = SGDC.predict(test)
print(y_pred[:20])
sol = pd.DataFrame(y_pred,columns=['slice Type'])
print(sol.head(20))


0.9993667880322938
              precision    recall  f1-score   support

           1       1.00      1.00      1.00     13439
           2       1.00      1.00      1.00      5913
           3       1.00      1.00      1.00      5914

    accuracy                           1.00     25266
   macro avg       1.00      1.00      1.00     25266
weighted avg       1.00      1.00      1.00     25266

              precision    recall  f1-score   support

           1       1.00      1.00      1.00      3360
           2       1.00      1.00      1.00      1479
           3       1.00      1.00      1.00      1478

    accuracy                           1.00      6317
   macro avg       1.00      1.00      1.00      6317
weighted avg       1.00      1.00      1.00      6317

[1 3 2 2 1 1 2 2 3 1 1 1 2 1 3 3 1 1 1 2]
    slice Type
0            1
1            3
2            2
3            2
4            1
5            1
6            2
7            2
8            3
9            1
10          

In [ ]:
from sklearn.svm import SVC

SVCLF= SVC(C=181.96184687258582, cache_size=1905.6666666666667,
      class_weight='balanced', gamma=1.8287341102334311, max_iter=-1.0,
      random_state=1, tol=0.013996997317851007)
SVCLF.fit(X_train,y_train)
y_pred=SVCLF.predict(X_test)
print(accuracy_score(y_test,y_pred))
print(classification_report(y_train, SVCLF.predict(X_train)))
print(classification_report(y_test, SVCLF.predict(X_test)))
y_pred = SVCLF.predict(test)
print(y_pred[:20])
sol = pd.DataFrame(y_pred,columns=['slice Type'])
print(sol.head(20))

1.0
              precision    recall  f1-score   support

           1       1.00      1.00      1.00     13439
           2       1.00      1.00      1.00      5913
           3       1.00      1.00      1.00      5914

    accuracy                           1.00     25266
   macro avg       1.00      1.00      1.00     25266
weighted avg       1.00      1.00      1.00     25266

              precision    recall  f1-score   support

           1       1.00      1.00      1.00      3360
           2       1.00      1.00      1.00      1479
           3       1.00      1.00      1.00      1478

    accuracy                           1.00      6317
   macro avg       1.00      1.00      1.00      6317
weighted avg       1.00      1.00      1.00      6317

[1 3 2 2 1 1 2 2 3 1 1 1 2 1 3 3 1 1 1 2]
    slice Type
0            1
1            3
2            2
3            2
4            1
5            1
6            2
7            2
8            3
9            1
10           1
11          

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
ETC=ExtraTreesClassifier(criterion='entropy', max_features=14, min_samples_leaf=2,
                       min_samples_split=6, n_estimators=512, n_jobs=1,
                       random_state=1, warm_start=True)
ETC.fit(X_train,y_train)
y_pred=ETC.predict(X_test)
print(accuracy_score(y_test,y_pred))
print(classification_report(y_train, ETC.predict(X_train)))
print(classification_report(y_test, ETC.predict(X_test)))
y_pred = ETC.predict(test)
print(y_pred[:20])
sol = pd.DataFrame(y_pred,columns=['slice Type'])
print(sol.head(20))

1.0
              precision    recall  f1-score   support

           1       1.00      1.00      1.00     13439
           2       1.00      1.00      1.00      5913
           3       1.00      1.00      1.00      5914

    accuracy                           1.00     25266
   macro avg       1.00      1.00      1.00     25266
weighted avg       1.00      1.00      1.00     25266

              precision    recall  f1-score   support

           1       1.00      1.00      1.00      3360
           2       1.00      1.00      1.00      1479
           3       1.00      1.00      1.00      1478

    accuracy                           1.00      6317
   macro avg       1.00      1.00      1.00      6317
weighted avg       1.00      1.00      1.00      6317

[1 3 2 2 1 1 2 2 3 1 1 1 2 1 3 3 1 1 1 2]
    slice Type
0            1
1            3
2            2
3            2
4            1
5            1
6            2
7            2
8            3
9            1
10           1
11          

In [ ]:
from sklearn.neural_network import MLPClassifier

MLPC= MLPClassifier(activation='tanh', alpha=3.542372581884717e-07, beta_1=0.999,
                beta_2=0.9, hidden_layer_sizes=(54, 54),
                learning_rate_init=0.00016753718126302934, max_iter=32,
                n_iter_no_change=32, random_state=1, validation_fraction=0.0,
                verbose=0, warm_start=True)
MLPC.fit(X_train,y_train)
y_pred=MLPC.predict(X_test)
print(accuracy_score(y_test,y_pred))
print(classification_report(y_train, MLPC.predict(X_train)))
print(classification_report(y_test, MLPC.predict(X_test)))
y_pred = MLPC.predict(test)
print(y_pred[:20])
sol = pd.DataFrame(y_pred,columns=['slice Type'])
print(sol.head(20))


0.9892353965489947
              precision    recall  f1-score   support

           1       1.00      1.00      1.00     13439
           2       1.00      0.96      0.98      5913
           3       0.96      1.00      0.98      5914

    accuracy                           0.99     25266
   macro avg       0.99      0.99      0.99     25266
weighted avg       0.99      0.99      0.99     25266

              precision    recall  f1-score   support

           1       1.00      1.00      1.00      3360
           2       1.00      0.96      0.98      1479
           3       0.96      1.00      0.98      1478

    accuracy                           0.99      6317
   macro avg       0.99      0.98      0.98      6317
weighted avg       0.99      0.99      0.99      6317

[1 3 2 2 1 1 3 2 3 1 1 1 2 1 3 3 1 1 1 2]
    slice Type
0            1
1            3
2            2
3            2
4            1
5            1
6            3
7            2
8            3
9            1
10          

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:617: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (32) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [15]:
from sklearn.linear_model import LogisticRegression


LRC=  LogisticRegression()
LRC.fit(X_train,y_train)
y_pred=LRC.predict(X_test)
print(accuracy_score(y_test,y_pred))
print(classification_report(y_train, LRC.predict(X_train)))
print(classification_report(y_test, LRC.predict(X_test)))
y_pred = LRC.predict(test)
print(y_pred[:20])
sol = pd.DataFrame(y_pred,columns=['slice Type'])
print(sol.head(20))


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


1.0
              precision    recall  f1-score   support

           1       1.00      1.00      1.00     13439
           2       1.00      1.00      1.00      5913
           3       1.00      1.00      1.00      5914

    accuracy                           1.00     25266
   macro avg       1.00      1.00      1.00     25266
weighted avg       1.00      1.00      1.00     25266

              precision    recall  f1-score   support

           1       1.00      1.00      1.00      3360
           2       1.00      1.00      1.00      1479
           3       1.00      1.00      1.00      1478

    accuracy                           1.00      6317
   macro avg       1.00      1.00      1.00      6317
weighted avg       1.00      1.00      1.00      6317

[1 3 2 2 1 1 2 2 3 1 1 1 2 1 3 3 1 1 1 2]
    slice Type
0            1
1            3
2            2
3            2
4            1
5            1
6            2
7            2
8            3
9            1
10           1
11          